In [3]:
#!pip3 install -U tensorflow

In [40]:
import os
import mne
import numpy as np
import tensorflow
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [8]:
all_files_path=glob('Downloads\dataverse_files\*.edf')


In [10]:
healthy_file_path=[i for i in all_files_path if 'h' in i.split('\\')[2]]
patient_file_path=[i for i in all_files_path if 's' in i.split('\\')[2]]
print(len(healthy_file_path),len(patient_file_path))

14 14


In [13]:
def read_data(file_path):
    data=mne.io.read_raw_edf(file_path,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [14]:
%%capture
control_epochs_array=[read_data(i) for i in healthy_file_path]
patient_epochs_array=[read_data(i) for i in patient_file_path]

In [19]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patient_epochs_labels=[len(i)*[1] for i in patient_epochs_array]

In [20]:
data_list=patient_epochs_array+control_epochs_array

In [21]:
label_list=control_epochs_labels+patient_epochs_labels

In [22]:
group_list=[[i]*len(j) for i,j in enumerate(data_list)]

In [23]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
print(data_array.shape,label_array.shape,group_array.shape)

(7201, 19, 1250) (7201,) (7201,)


features of data

In [24]:
from scipy import stats
def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)


def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [25]:
features=[]
for d in data_array:
    features.append(concatenate_features(d))

In [26]:
features_array=np.array(features)

# classification through ml

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate

In [55]:
model=LogisticRegression()
gkf=GroupKFold(5)
pipe=Pipeline([('scaler',StandardScaler()),('clf',model)])
param_grid={'clf__C':[0.1,0.3,0.5,0.7,0.8,1.0,3,5,6,7,8,11,13,15,20]}
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=2)
gscv.fit(features,label_array,groups=group_array)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', LogisticRegression())]),
             n_jobs=2,
             param_grid={'clf__C': [0.1, 0.3, 0.5, 0.7, 0.8, 1.0, 3, 5, 6, 7, 8,
                                    11, 13, 15, 20]})

In [56]:
accuracy=gscv.best_score_*100
print(accuracy)

62.337295204929745


# classification through dl

In [32]:
data_array=np.moveaxis(data_array,2,1)#keras cnn expects channels as third parameter
data_array.shape

(7201, 1250, 19)

In [33]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

In [34]:

def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(1250,19)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1248, 5)           290       
                                                                 
 batch_normalization (BatchN  (None, 1248, 5)          20        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1248, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 624, 5)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 622, 5)            80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 622, 5)            0

In [35]:

gkf=GroupKFold()

In [65]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=12,batch_size=128,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])
    

Epoch 1/12
45/45 [==============================] - 3s 48ms/step - loss: 0.6861 - accuracy: 0.5437 - val_loss: 0.8245 - val_accuracy: 0.0000e+00
Epoch 2/12
45/45 [==============================] - 2s 43ms/step - loss: 0.6743 - accuracy: 0.5660 - val_loss: 0.8236 - val_accuracy: 0.0000e+00
Epoch 3/12
45/45 [==============================] - 2s 43ms/step - loss: 0.6628 - accuracy: 0.5660 - val_loss: 0.7876 - val_accuracy: 0.0110
Epoch 4/12
45/45 [==============================] - 2s 43ms/step - loss: 0.6313 - accuracy: 0.6196 - val_loss: 0.7328 - val_accuracy: 0.7351
Epoch 5/12
45/45 [==============================] - 2s 43ms/step - loss: 0.5352 - accuracy: 0.7679 - val_loss: 0.5723 - val_accuracy: 0.8037
Epoch 6/12
45/45 [==============================] - 2s 45ms/step - loss: 0.4754 - accuracy: 0.7853 - val_loss: 0.8748 - val_accuracy: 0.7392
Epoch 7/12
45/45 [==============================] - 2s 49ms/step - loss: 0.4356 - accuracy: 0.8134 - val_loss: 0.7896 - val_accuracy: 0.7831
Epoch

In [66]:
np.mean(accuracy)

0.6071841776371002